In [ ]:
from osgeo import gdal
import numpy as np
import os

def reclassify_dem(input_path, output_path):
    if not os.path.exists(input_path):
        print(f"错误：找不到输入文件 {input_path}")
        return

    ds = gdal.Open(input_path, gdal.GA_ReadOnly)
    if ds is None:
        print("无法打开文件")
        return

    band = ds.GetRasterBand(1)
    data = band.ReadAsArray()
    
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    x_size = ds.RasterXSize
    y_size = ds.RasterYSize

    result = np.full(data.shape, 255, dtype=np.uint8)
    
    result[(data >= 2200) & (data < 2300)] = 0
    result[(data >= 1000) & (data < 1900)] = 1 
    result[(data >= 2000) & (data < 2200)] = 2
    result[data < 1000] = 3
    result[(data >= 2800) & (data < 3000)] = 4
    result[(data >= 3100) & (data < 3400)] = 5
    result[(data >= 3400) & (data < 3600)] = 6
    result[data >= 3600] = 7
    result[(data >= 1900) & (data < 2000)] = 8
    result[(data >= 2300) & (data < 2500)] = 9
    result[(data >= 2500) & (data < 2700)] = 10
    result[(data >= 2700) & (data < 2800)] = 11
    result[(data >= 3000) & (data < 3100)] = 12

    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_path, x_size, y_size, 1, gdal.GDT_Byte)
    
    if out_ds is None:
        print(f"无法创建输出文件 {output_path}")
        return

    out_ds.SetGeoTransform(geotransform)
    out_ds.SetProjection(projection)
    
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(result)
    
    out_band.SetNoDataValue(255)
    
    out_band.FlushCache()
    out_ds = None
    ds = None
    
    print(f"处理完成！0值已被视为有效分类。文件保存至: {output_path}")

if __name__ == "__main__":
    input_file = r"D:\Desktop\ResNet50_Paper\Review_comments\Exp\Exp2\Predict_Data\DEM.tif"
    output_file = r"D:\Desktop\ResNet50_Paper\Review_comments\Exp\Exp1\DEM_MVKG_result.tif"
    reclassify_dem(input_file, output_file)